<a href="https://colab.research.google.com/github/GU-DPM/EvolutionaryClassifier/blob/main/analysis/EC_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

if running on colab...

In [1]:
from google.colab import auth
from google.colab import data_table
data_table.enable_dataframe_formatter()
auth.authenticate_user()

otherwise just start here if running from VM...

In [2]:
from google.cloud import bigquery
project = 'mccoylab' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)

In [3]:
import pandas as pd
import seaborn as sns

Define benefit query

In [18]:
# build benefit query
# firstSame, secondSame, bothDiff, bothSame
benefit_def = 'secondSame'
num_records = 50

#benefit_query = """
#SELECT *
#  FROM `mccoylab.dpm_dev.dpmTrial2019_benefitSummary` dpm
# WHERE dpm.Category = \'""" + benefit_def  + """\'
#ORDER BY RAND() LIMIT """ + str(num_records) + """
#"""

#collect sample population from whole dataset
#benefit_query = """
#SELECT *
#  FROM `mccoylab.dpm_dev.dpmTrial2019_benefitSummary` dpm
#ORDER BY RAND() LIMIT """ + str(num_records) + """
#"""

#collect sample population from patients that benefit on trial
#benefit_query = """
#SELECT *
#  FROM `mccoylab.dpm_dev.dpmTrial2019_benefitSummary` dpm
# WHERE dpm.improvementDPMtrial is true
#ORDER BY RAND() LIMIT """ + str(num_records) + """
#"""





benefit_query

'\nSELECT *\n  FROM `mccoylab.dpm_dev.dpmTrial2019_benefitSummary` dpm\n WHERE dpm.improvementDPMtrial is true\nORDER BY RAND() LIMIT 50\n'

In [19]:
# sample results for virtual patients that neet benefit definiton

benefit_df = client.query(benefit_query).to_dataframe()
benefit_df

,int64_field_0,paramID,Category,Surv_0,Surv_2_2,Surv_2_2trial,Drug1Step1_0,Drug2Step1_0,Drug1Step2_0,Drug2Step2_0,Drug1Step1_2_2,Drug2Step1_2_2,Drug1Step2_2_2,Drug2Step2_2_2,improvementDays_DPM,improvementPercent_DPM,improvementDays_DPMtrial,improvementPercent_DPMtrial,improvementDPM,improvementDPMtrial
0,1570361,23052505,bothDiff,90,1845,1845,0,1,1,0,0.5,0.5,0.0,1.0,1755,19.500000,1755,19.500000,True,True
1,2173523,30213422,bothDiff,945,1845,1845,0,1,0,1,1.0,0.0,1.0,0.0,900,0.952381,900,0.952381,True,True
2,770110,12936691,secondSame,675,1845,1845,0,1,0,1,1.0,0.0,0.0,1.0,1170,1.733333,1170,1.733333,True,True
3,2370651,32107105,bothDiff,0,90,90,0,1,-1,-1,1.0,0.0,0.5,0.5,90,inf,90,inf,True,True
4,2893524,37763999,bothDiff,180,1845,1845,1,0,1,0,0.5,0.5,0.0,1.0,1665,9.250000,1665,9.250000,True,True
5,1797346,26096527,secondSame,675,1845,1845,0,1,0,1,1.0,0.0,0.0,1.0,1170,1.733333,1170,1.733333,True,True
6,36847,821539,bothDiff,45,1845,1845,1,0,1,0,0.0,1.0,0.0,1.0,1800,40.000000,1800,40.000000,True,True
7,2885572,37746135,bothDiff,270,1845,1845,1,0,1,0,0.5,0.5,0.0,1.0,1575,5.833333,1575,5.833333,True,True
8,2508733,33642917,bothDiff,270,1845,1845,0,1,1,0,0.5,0.5,0.0,1.0,1575,5.833333,1575,5.833333,True,True
9,967717,15612208,bothDiff,0,1845,1845,1,0,-1,-1,0.5,0.5,0.0,1.0,1845,inf,1845,inf,True,True


In [9]:
# build state variable vector
paramID_list = ",".join(str(x) for x in benefit_df.paramID.tolist())

stateVar_query = """
SELECT pop.parameter_index, pop.strategy_name,
       pop.s_t_0, pop.r1_t_0, pop.r2_t_0, pop.r12_t_0,
       pop.s_t_45, pop.r1_t_45, pop.r2_t_45, pop.r12_t_45,
 FROM `mccoylab.dpm_dev.pop-results-2019` pop
 WHERE pop.parameter_index IN (""" + paramID_list + """)
"""
stateVar_query

'\nSELECT pop.parameter_index, pop.strategy_name,\n       pop.s_t_0, pop.r1_t_0, pop.r2_t_0, pop.r12_t_0,\n       pop.s_t_45, pop.r1_t_45, pop.r2_t_45, pop.r12_t_45,\n FROM `mccoylab.dpm_dev.pop-results-2019` pop\n WHERE pop.parameter_index IN (23034022,27876071,3006527,7271268,34560328,13135956,18654825,20536176,18103583,26112099,7290258,1360409,19612425,584316,21865120,795506,3007995,3243542,23843328,33374449,38563851,15627238,32812564,37639051,19744953,16300798,33754195,37832062,37742972,24667100,18686084,21360079,25391400,37612387,31137508,15526011,29492396,18584446,9035252,28452304,37869810,22197312,30229821,14805411,32904064,12332988,34448038,9829697,28672068,33645328)\n'

In [10]:
stateVar_df = client.query(stateVar_query).to_dataframe()
stateVar_df

,parameter_index,strategy_name,s_t_0,r1_t_0,r2_t_0,r12_t_0,s_t_45,r1_t_45,r2_t_45,r12_t_45
0,9829697,0.0,1.1087e+13,1.2794e+04,2.2068e+13,1.4117e+03,-1.0000e+00,-1.0000e+00,-1.0000e+00,-1.0000e+00
1,9829697,2.0,1.1087e+13,2.2507e+07,1.1115e+10,4.8967e+00,-1.0000e+00,-1.0000e+00,-1.0000e+00,-1.0000e+00
2,9829697,1.0,1.1087e+13,2.2507e+07,1.1115e+10,4.8967e+00,-1.0000e+00,-1.0000e+00,-1.0000e+00,-1.0000e+00
3,7290258,1.0,9.9990e-01,9.9998e-01,9.9997e-01,2.6529e-06,9.9990e-01,9.9998e-01,9.9997e-01,2.6529e-06
4,7290258,2.0,9.9990e-01,9.9998e-01,9.9997e-01,2.6529e-06,9.9990e-01,9.9998e-01,9.9997e-01,2.6529e-06
...,...,...,...,...,...,...,...,...,...,...
145,23843328,1.0,1.0189e+14,1.0215e+11,2.8980e+12,2.9054e+09,-1.0000e+00,-1.0000e+00,-1.0000e+00,-1.0000e+00
146,23843328,2.0,1.0189e+14,1.0215e+11,2.8980e+12,2.9054e+09,-1.0000e+00,-1.0000e+00,-1.0000e+00,-1.0000e+00
147,21865120,1.0,9.9988e-01,2.4535e+05,9.9997e-01,1.8120e+04,9.9988e-01,9.9998e-01,9.9997e-01,2.9817e+04
148,21865120,2.0,9.9988e-01,2.4535e+05,9.9997e-01,1.8120e+04,9.9988e-01,9.9998e-01,9.9997e-01,2.9817e+04


In [11]:
# get initial param
param_query = """
SELECT *
  FROM `mccoylab.dpm_dev.param-results-2019` param
 WHERE param.parameter_index IN (""" + paramID_list + """)
"""
param_query

'\nSELECT *\n  FROM `mccoylab.dpm_dev.param-results-2019` param\n WHERE param.parameter_index IN (23034022,27876071,3006527,7271268,34560328,13135956,18654825,20536176,18103583,26112099,7290258,1360409,19612425,584316,21865120,795506,3007995,3243542,23843328,33374449,38563851,15627238,32812564,37639051,19744953,16300798,33754195,37832062,37742972,24667100,18686084,21360079,25391400,37612387,31137508,15526011,29492396,18584446,9035252,28452304,37869810,22197312,30229821,14805411,32904064,12332988,34448038,9829697,28672068,33645328)\n'

In [12]:
param_df = client.query(param_query).to_dataframe()
param_df

,parameter_index,s_init_pop,r1_init_pop,r2_init_pop,r12_init_pop,growth_rate,s_drug1_sensitivity,s_drug2_sensitivity,r1_drug1_sensitivity,r1_drug2_sensitivity,...,r1_r2_transition_rate,r1_r12_transition_rate,r2_s_transition_rate,r2_r1_transition_rate,r2_r2_transition_rate,r2_r12_transition_rate,r12_s_transition_rate,r12_r1_transition_rate,r12_r2_transition_rate,r12_r12_transition_rate
0,32812564,4.495000e+09,5.000000e+08,5.000000e+06,0.0,0.1287,0.305790,56.628000,0.305790,5.662800e-04,...,0.0,0.0,2.154000e-10,0.0,0.0,0.0,0.0,2.154000e-10,2.154000e-10,0.0
1,32904064,4.495000e+09,5.000000e+08,5.000000e+06,0.0,0.3400,0.440000,1.621200,0.440000,1.550500e-04,...,0.0,0.0,2.154000e-06,0.0,0.0,0.0,0.0,2.154000e-06,1.000000e-07,0.0
2,13135956,5.000000e+09,5.000000e+02,5.000000e+00,0.0,0.3400,0.119480,1.621200,0.119480,0.000000e+00,...,0.0,0.0,2.154000e-10,0.0,0.0,0.0,0.0,2.154000e-10,4.642000e-05,0.0
3,23843328,4.995000e+09,5.000000e+06,0.000000e+00,0.0,0.3400,0.119480,1.621200,0.119480,1.410500e-02,...,0.0,0.0,4.642000e-05,0.0,0.0,0.0,0.0,4.642000e-05,1.000000e-07,0.0
4,24667100,4.995000e+09,5.000000e+06,5.000000e+00,0.0,0.3400,0.119480,1.621200,0.119480,1.410500e-02,...,0.0,0.0,1.000000e-07,0.0,0.0,0.0,0.0,1.000000e-07,2.154000e-10,0.0
5,23034022,4.999500e+08,5.000000e+04,4.500000e+09,0.0,0.3400,0.311470,15.574000,0.311470,1.424400e-02,...,0.0,0.0,2.154000e-06,0.0,0.0,0.0,0.0,2.154000e-06,1.000000e-11,0.0
6,18103583,5.000000e+09,5.000000e+04,0.000000e+00,0.0,0.3400,0.224400,149.600000,0.224400,0.000000e+00,...,0.0,0.0,2.154000e-10,0.0,0.0,0.0,0.0,2.154000e-10,1.000000e-03,0.0
7,18584446,4.999900e+09,5.000000e+04,5.000000e+00,0.0,0.0184,2.197300,8.096000,2.197300,7.742600e-04,...,0.0,0.0,1.000000e-03,0.0,0.0,0.0,0.0,1.000000e-03,4.642000e-09,0.0
8,31137508,4.500000e+09,5.000000e+08,5.000000e+02,0.0,0.1287,0.045228,0.613680,0.045228,5.612800e-04,...,0.0,0.0,1.000000e-07,0.0,0.0,0.0,0.0,1.000000e-07,4.642000e-09,0.0
9,38563851,4.950000e+08,4.500000e+09,5.000000e+06,0.0,0.1287,0.117900,5.895000,0.117900,5.128700e-02,...,0.0,0.0,2.154000e-06,0.0,0.0,0.0,0.0,2.154000e-06,4.642000e-09,0.0


In [15]:
benefit_df.to_csv("benefit_summary.csv", header=True, index=False)
stateVar_df.to_csv("statvar.csv", header=True, index=False)
param_df.to_csv("inputParam.csv", header=True, index=False)

In [ ]:
# generate state variable features



In [ ]:
# kmeans clustering ?

In [ ]:
# sample centroids and plot trajectories

In [ ]:
# heirarchal clustering and dendrogram (color by kmeans?)

In [ ]:
# survival plot
